In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

BASE_URL   = "https://www.thegioididong.com/dtdd#c=42&o=13&pi={}"
TIMEOUT    = 15
MAX_PAGES  = 50  # Giới hạn tối đa để tránh infinite loop

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
driver = webdriver.Chrome(options=options)

def scroll_to_bottom(driver):
    """Scroll từ từ để load hết sản phẩm"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Tăng thời gian chờ
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

all_links = []

def detect_last_page(driver):
    """Detect xem có phải trang cuối không"""
    try:
        # Kiểm tra không có sản phẩm nào
        products = driver.find_elements(By.CSS_SELECTOR, "ul.listproduct li")
        if len(products) == 0:
            return True
        return False
    except Exception:
        return False

all_links = []
page = 1
previous_total_count = 0

while page <= MAX_PAGES:
    print(f"Scraping page {page}...")
    
    # Điều hướng đến trang
    driver.get(BASE_URL.format(page))
    
    # Chờ trang load xong
    time.sleep(3)
    
    # REFRESH/RELOAD trang như bấm Ctrl + R
    driver.refresh()
    
    # Chờ sau khi refresh
    time.sleep(3)
    
    try:
        # Scroll để load hết sản phẩm
        scroll_to_bottom(driver)
        
        # Chờ elements xuất hiện
        wait = WebDriverWait(driver, TIMEOUT)
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.listproduct li")))
        
        # Tìm tất cả sản phẩm
        lis = driver.find_elements(By.CSS_SELECTOR, "ul.listproduct li")
        print(f"Found {len(lis)} products on page {page}")
        
        # Nếu không có sản phẩm nào, dừng lại
        if len(lis) == 0:
            print(f"No products found on page {page}. Stopping...")
            break
        
        # Thêm sản phẩm vào danh sách
        valid_products = 0
        for i, li in enumerate(lis, 1):
            try:
                # Thử nhiều selector khác nhau cho link
                a = None
                for selector in ["a", "a[href]", ".product-link", "[href]"]:
                    a_elements = li.find_elements(By.CSS_SELECTOR, selector)
                    if a_elements:
                        a = a_elements[0]
                        break
                
                if not a:
                    continue
                
                # Thử nhiều selector khác nhau cho title
                h3 = None
                title = ""
                for selector in ["h3", ".product-title", ".title", "h3 a", "a"]:
                    h3_elements = li.find_elements(By.CSS_SELECTOR, selector)
                    if h3_elements:
                        h3 = h3_elements[0]
                        title = h3.text.strip()
                        if title:
                            break
                
                if not title:
                    continue
                
                href = a.get_attribute("href")
                
                if href and title:
                    all_links.append({"href": href, "title": title})
                    valid_products += 1
                    
            except Exception as e:
                # Chỉ log lỗi nếu debug mode
                # print(f"Skipping product {i}: {e}")
                continue
        
        print(f"Valid products processed: {valid_products}/{len(lis)}")
        
        # Lọc trùng lặp tạm thời để đếm chính xác
        unique_links_temp = []
        seen = set()
        for link in all_links:
            key = (link["href"], link["title"])
            if key not in seen:
                seen.add(key)
                unique_links_temp.append(link)
        
        current_total_count = len(unique_links_temp)
        print(f"Current total unique products: {current_total_count}")
        
        # KIỂM TRA: Nếu số lượng không tăng lên, dừng lại
        if current_total_count == previous_total_count:
            print(f"No new products found on page {page}. Total count remained at {current_total_count}. Stopping...")
            break
        
        # Cập nhật count cho lần kiểm tra tiếp theo
        previous_total_count = current_total_count
                
    except Exception as e:
        print(f"Error on page {page}: {e}")
        break
    
    # Nghỉ giữa các page để tránh bị block
    time.sleep(2)
    page += 1

print(f"Finished scraping at page {page - 1}")

driver.quit()

# Lọc trùng lặp
unique_links = []
seen = set()

for link in all_links:
    key = (link["href"], link["title"])
    if key not in seen:
        seen.add(key)
        unique_links.append(link)

print(f"Collected {len(unique_links)} unique products")
print("\nFirst 5 products:")
for i, link in enumerate(unique_links[:5], 1):
    print(f"{i}. {link['title']}")
    print(f"   URL: {link['href']}")
    print()

Scraping page 1...
Found 123 products on page 1
Valid products processed: 40/123
Current total unique products: 40
Scraping page 2...
Found 181 products on page 2
Valid products processed: 60/181
Current total unique products: 60
Scraping page 3...
Found 227 products on page 3
Valid products processed: 79/227
Current total unique products: 79
Scraping page 4...
Found 253 products on page 4
Valid products processed: 99/253
Current total unique products: 99
Scraping page 5...
Found 291 products on page 5
Valid products processed: 119/291
Current total unique products: 119
Scraping page 6...
Found 309 products on page 6
Valid products processed: 131/309
Current total unique products: 131
Scraping page 7...
Found 309 products on page 7
Valid products processed: 131/309
Current total unique products: 131
No new products found on page 7. Total count remained at 131. Stopping...
Finished scraping at page 6
Collected 131 unique products

First 5 products:
1. OPPO Reno14 F 5G 12GB/256GB
   URL:

In [2]:
len(unique_links)

131

loop qua mỗi `unique_links`, lấy những thông số cần thiết như màn hình, pin, camera,...

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# ✅ Chuyển hết về lowercase để so khớp
ALLOWED_KEYS = {
    "công nghệ màn hình",
    "độ phân giải màn hình",
    "màn hình rộng",
    "hệ điều hành",
    "độ phân giải camera sau",
    "độ phân giải camera trước",
    "chip xử lý (cpu)",
    "ram",
    "dung lượng lưu trữ",
    "sim",
    "mạng di động",
    "dung lượng pin",
    "hỗ trợ sạc tối đa",
    "hãng"
}

product_details = []

for i, link in enumerate(unique_links, 1):
    url = link["href"]
    title = link["title"]
    print(f"({i}/{len(unique_links)}) Crawling details for: {title}")

    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.encoding = "utf-8"  # ✅ Sửa lỗi encoding
        soup = BeautifulSoup(r.text, "html.parser")

        specs = {}

        # ✅ Loại 1 (chỉ lưu lại các key được phép)
        items = soup.select("div.item.cf-right ul.parameter li")
        if items:
            for li in items:
                span = li.find("span")
                div = li.find("div")
                if span and div:
                    key = span.text.strip().lower().replace(":", "")
                    value = div.text.strip().replace(". Xem thông tin hãng", "")
                    specs[key] = value
                    # if key in ALLOWED_KEYS and value:
                    #     specs[key] = value
        else:
            # ✅ Loại 2 (lọc theo ALLOWED_KEYS)
            boxes = soup.select("div.specification-item div.box-specifi ul li")
            for li in boxes:
                asides = li.find_all("aside")
                if len(asides) >= 2:
                    key = asides[0].text.strip().lower().replace(":", "")
                    value = asides[1].text.strip().replace(". Xem thông tin hãng", "")
                    if key in ALLOWED_KEYS and value:
                        specs[key] = value

        product_details.append({
            "title": title,
            "url": url,
            "specs": specs
        })

        time.sleep(1)  # tránh bị block

    except Exception as e:
        print(f"❌ Error crawling {url}: {e}")
        continue

print(f"\n✅ Crawled details for {len(product_details)} products.")

# ✅ Lưu ra CSV (dùng <br> để ngắt dòng các specs)
with open("products_details.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "url", "specifications"])
    for p in product_details:
        specs_str = "<br>".join([f"{k}: {v}" for k, v in p["specs"].items()])
        writer.writerow([p["title"], p["url"], specs_str])

print("✅ Saved to products_details.csv")

In [3]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import re

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# ✅ Chuyển hết về lowercase để so khớp
ALLOWED_KEYS = {
    "công nghệ màn hình",
    "độ phân giải màn hình",
    "màn hình rộng",
    "hệ điều hành",
    "độ phân giải camera sau",
    "độ phân giải camera trước",
    "chip xử lý (cpu)",
    "ram",
    "dung lượng lưu trữ",
    "sim",
    "mạng di động",
    "dung lượng pin",
    "hỗ trợ sạc tối đa",
    "hãng"
}

def extract_price(soup):
    """
    Trích xuất giá sản phẩm từ các định dạng khác nhau
    """
    price = None
    
    # Loại 1: <div class="item cf-left"> -> <div> -> <b>Giá bán: <b>10.000.000<b/><b/>
    price_items = soup.select("div.item.cf-left")
    for item in price_items:
        b_tags = item.find_all("b")
        for b_tag in b_tags:
            text = b_tag.get_text().strip().lower()
            if "giá bán" in text or "giá" in text:
                # Tìm giá trong các thẻ b tiếp theo
                parent = b_tag.parent
                if parent:
                    price_text = parent.get_text()
                    price_match = re.search(r'[\d,.\s]+', price_text.replace("Giá bán:", "").strip())
                    if price_match:
                        price = price_match.group().strip()
                        break
        if price:
            break
    
    # Loại 2 - Kiểu 1: <div class="price-one"> -> <div class="box-price"> -> <p class="box-price-present">
    if not price:
        price_one = soup.select_one("div.price-one div.box-price p.box-price-present")
        if price_one:
            price = price_one.get_text().strip()
    
    # Loại 2 - Kiểu 2: <div class="bs_price"> -> <strong>1.000.000</strong>
    if not price:
        bs_price = soup.select_one("div.bs_price strong")
        if bs_price:
            price = bs_price.get_text().strip()
    
    # Thêm các pattern khác có thể có
    if not price:
        # Tìm trong các class phổ biến khác
        common_price_selectors = [
            ".price-current",
            ".price-new",
            ".price-main",
            ".product-price",
            ".price-box strong",
            ".price-present",
            "span.price",
            ".current-price"
        ]
        
        for selector in common_price_selectors:
            price_elem = soup.select_one(selector)
            if price_elem:
                price = price_elem.get_text().strip()
                break
    
    # Làm sạch giá (loại bỏ ký tự không cần thiết)
    if price:
        price = re.sub(r'[^\d,.\s₫]', '', price).strip()
        price = price.replace('₫', '').strip()
    
    return price

def categorize_price(price_str):
    """
    Phân loại mức giá để hỗ trợ tư vấn
    """
    if not price_str:
        return "unknown"
    
    # Chuyển giá về số (loại bỏ dấu phẩy, chấm)
    price_clean = re.sub(r'[,.\s]', '', price_str)
    try:
        price_num = int(price_clean)
        if price_num < 5000000:
            return "thap"  # Dưới 5 triệu
        elif price_num < 10000000:
            return "trung"  # 5-10 triệu
        elif price_num < 20000000:
            return "trung_cao"  # 10-20 triệu
        else:
            return "cao"  # Trên 20 triệu
    except:
        return "unknown"

def extract_key_features(specs):
    """
    Trích xuất các tính năng nổi bật để hỗ trợ tư vấn
    """
    features = []
    
    # Camera
    if "độ phân giải camera sau" in specs:
        camera_value = specs["độ phân giải camera sau"]
        if "50" in camera_value or "48" in camera_value:
            features.append("camera chất lượng cao")
    
    # Pin
    if "dung lượng pin" in specs:
        battery_value = specs["dung lượng pin"]
        battery_match = re.search(r'(\d+)', battery_value)
        if battery_match:
            battery_mah = int(battery_match.group(1))
            if battery_mah >= 5000:
                features.append("pin khủng")
    
    # RAM
    if "ram" in specs:
        ram_value = specs["ram"]
        if "12" in ram_value or "16" in ram_value:
            features.append("RAM lớn")
    
    # Sạc nhanh
    if "hỗ trợ sạc tối đa" in specs:
        charging_value = specs["hỗ trợ sạc tối đa"]
        charging_match = re.search(r'(\d+)', charging_value)
        if charging_match:
            charging_w = int(charging_match.group(1))
            if charging_w >= 60:
                features.append("sạc siêu nhanh")
    
    return features

product_details = []

for i, link in enumerate(unique_links, 1):
    url = link["href"]
    title = link["title"]
    print(f"({i}/{len(unique_links)}) Crawling details for: {title}")

    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.encoding = "utf-8"  # ✅ Sửa lỗi encoding
        soup = BeautifulSoup(r.text, "html.parser")

        specs = {}
        
        # ✅ Trích xuất giá sản phẩm
        price = extract_price(soup)
        price_category = categorize_price(price)

        # ✅ Loại 1 (chỉ lưu lại các key được phép)
        items = soup.select("div.item.cf-right ul.parameter li")
        if items:
            for li in items:
                span = li.find("span")
                div = li.find("div")
                if span and div:
                    key = span.text.strip().lower().replace(":", "")
                    value = div.text.strip().replace(". Xem thông tin hãng", "")
                    specs[key] = value
        else:
            # ✅ Loại 2 (lọc theo ALLOWED_KEYS)
            boxes = soup.select("div.specification-item div.box-specifi ul li")
            for li in boxes:
                asides = li.find_all("aside")
                if len(asides) >= 2:
                    key = asides[0].text.strip().lower().replace(":", "")
                    value = asides[1].text.strip().replace(". Xem thông tin hãng", "")
                    if key in ALLOWED_KEYS and value:
                        specs[key] = value

        # ✅ Trích xuất tính năng nổi bật
        key_features = extract_key_features(specs)
        
        # ✅ Xác định nhóm người dùng mục tiêu
        target_users = []
        if "camera chất lượng cao" in key_features:
            target_users.append("chụp ảnh")
        if "RAM lớn" in key_features:
            target_users.append("gaming")
        if "pin khủng" in key_features:
            target_users.append("sử dụng lâu")
        if not target_users:
            target_users.append("sử dụng cơ bản")

        product_details.append({
            "title": title,
            "url": url,
            "price": price or "Liên hệ",
            "price_category": price_category,
            "key_features": key_features,
            "target_users": target_users,
            "specs": specs
        })

        time.sleep(1)  # tránh bị block

    except Exception as e:
        print(f"❌ Error crawling {url}: {e}")
        continue

print(f"\n✅ Crawled details for {len(product_details)} products.")

# ✅ Lưu ra CSV với thông tin mở rộng
with open("products_details_enhanced.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "url", "price", "price_category", "key_features", "target_users", "specifications"])
    for p in product_details:
        specs_str = "<br>".join([f"{k}: {v}" for k, v in p["specs"].items()])
        key_features_str = ", ".join(p["key_features"])
        target_users_str = ", ".join(p["target_users"])
        writer.writerow([
            p["title"], 
            p["url"], 
            p["price"], 
            p["price_category"],
            key_features_str,
            target_users_str,
            specs_str
        ])

print("✅ Saved to products_details_enhanced.csv")

# ✅ Thống kê nhanh
print("\n📊 THỐNG KÊ:")
price_stats = {}
for p in product_details:
    category = p["price_category"]
    price_stats[category] = price_stats.get(category, 0) + 1

for category, count in price_stats.items():
    print(f"  - {category}: {count} sản phẩm")

print(f"\n🔍 Ví dụ dữ liệu:")
if product_details:
    sample = product_details[0]
    print(f"  - Tên: {sample['title']}")
    print(f"  - Giá: {sample['price']}")
    print(f"  - Mức giá: {sample['price_category']}")
    print(f"  - Tính năng: {sample['key_features']}")
    print(f"  - Người dùng: {sample['target_users']}")

(1/131) Crawling details for: OPPO Reno14 F 5G 12GB/256GB
(2/131) Crawling details for: Samsung Galaxy Z Flip7 FE 5G 8GB/128GB
(3/131) Crawling details for: iPhone 16 Pro Max 256GB
(4/131) Crawling details for: iPhone 16 Pro 128GB
(5/131) Crawling details for: Samsung Galaxy Z Fold7 5G 12GB/256GB
(6/131) Crawling details for: Samsung Galaxy Z Flip7 5G 12GB/256GB
(7/131) Crawling details for: OPPO Reno14 5G 12GB/512GB
(8/131) Crawling details for: OPPO Reno14 Pro 5G 12GB/512GB
(9/131) Crawling details for: Xiaomi Redmi Note 14 Pro 8GB/256GB
(10/131) Crawling details for: Xiaomi Redmi Note 14 Pro+ 5G 8GB/256GB
(11/131) Crawling details for: vivo Y39 5G 8GB/128GB
(12/131) Crawling details for: vivo V50 Lite 5G 8GB/256GB
(13/131) Crawling details for: realme 14T 5G 8GB/256GB
(14/131) Crawling details for: HONOR 400 5G 12GB/256GB Đen
(15/131) Crawling details for: Tecno Spark 30 8GB/128GB
(16/131) Crawling details for: Masstel S9 6GB/256GB
(17/131) Crawling details for: iPhone 16 Plus 128GB